#**Intégration d'Ollama pour la génération des réponses et Développement de l'interface utilisateur**

##**Introduction**

Ce notebook met en place un assistant de recherche scientifique capable de répondre à des questions en fonction du profil utilisateur (étudiant ou chercheur). Il utilise un paradigme basé sur les embeddings pour trouver les paragraphes les plus pertinents dans une base de données scientifique stockée dans Chroma, et génère des réponses personnalisées grâce à Ollama, un modèle de langage pré-entraîné. Le tout est intégré dans une interface simple via Streamlit pour permettre une interaction directe avec l’utilisateur.

___________________________________

###**1. Installation des bibliothèques nécessaires**

Dans cette première étape, nous installons les bibliothèques qui seront utilisées tout au long du notebook :

- LangChain et LangChain Community : Ces bibliothèques permettent d'exploiter les capacités avancées des modèles de langage (LLM).
- Chroma : Un outil de base de données vectorielle pour stocker et interroger les embeddings des textes.
- Transformers : Utilisé pour charger des modèles de langage comme DistilBERT pour générer des embeddings.
- Requests : Cette bibliothèque facilite la communication avec l'API locale d'Ollama.

In [ ]:
!pip install langchain chromadb langchain_community transformers requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.6/602.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.3 MB/s eta 0:00:00
 

###**2. Importation des bibliothèques et configuration de l’environnement**

Nous importons ici les bibliothèques et modules indispensables à la construction de notre assistant :

- Streamlit : Utilisé pour créer l'interface utilisateur.
- Chroma : Pour interroger la base de données contenant les embeddings.
- Ollama : Un LLM qui sera utilisé pour générer des réponses basées sur le contexte.
- DistilBERT : Chargé de transformer les requêtes en embeddings qui seront utilisés par Chroma.

Ces bibliothèques assurent une communication fluide entre le back-end (modèles de langage et base de données) et le front-end (interface utilisateur).

In [ ]:
import streamlit as st
from flask import Flask, render_template, request
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain.vectorstores import Chroma

In [ ]:
import chromadb
import pandas as pd
import re
import requests
from transformers import DistilBertTokenizer, DistilBertModel

###**3. Initialisation de Chroma et chargement du modèle**

Nous configurons ici un client Chroma qui se connecte à un répertoire persistant contenant les embeddings des articles scientifiques. La collection "Article_Embedding" contient les vecteurs représentant chaque paragraphe des articles, qui seront utilisés plus tard pour la recherche de similarités avec les requêtes des utilisateurs.

In [ ]:
# Indiquer le chemin vers le répertoire de Chroma dans Google Drive
CHROMA_PATH = "chroma_db_persistent"

# Initialiser le client Chroma avec le répertoire persistant
client = chromadb.PersistentClient(path=CHROMA_PATH) # Set the persist directory here

# Create or get the collection, persist_directory is no longer needed.
collection = client.get_or_create_collection("Article_Embedding")

###**4. Chargement du modèle DistilBERT**

Nous utilisons DistilBERT, un modèle de langage compact et performant. Son rôle est de convertir le texte des requêtes en vecteurs de caractéristiques (embeddings) pour permettre à Chroma de trouver les sections de texte les plus pertinentes.

In [ ]:
# Charger les modèles
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

###**5 : Fonction de génération des embeddings**

La fonction *get_distilbert_embeddings* transforme le texte en un vecteur numérique (embedding) à l'aide de DistilBERT. Ce vecteur est une représentation condensée du texte, qui capture le sens global de la requête de l'utilisateur. Il est essentiel pour interroger la base de données Chroma et trouver des passages similaires.

In [ ]:
def get_distilbert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().reshape(-1)  # Reshape to 1D

###**6. Mise en place du modèle de prompt pour Ollama**

Ce template de prompt définit la structure des questions envoyées à Ollama. Nous y intégrons :

- Le profil de l'utilisateur (étudiant ou chercheur) pour personnaliser la réponse.
- Le contexte extrait de Chroma, qui contient les sections de texte les plus pertinentes.
- La question de l'utilisateur pour générer une réponse spécifique.

In [ ]:
PROMPT_TEMPLATE = """
You are a chatbot assistant who answers scientific questions.
Your responses should be tailored based on the user's profile and in English.

User profile: {profile}

Answer the question based on the following context and take into account the user's profile:

{context}

---

Answer the question based on the above context and profile: {question}
"""

###**7. Extraction du contexte pertinent depuis Chroma**

Cette fonction interroge Chroma pour récupérer les 10 documents les plus pertinents en fonction de la similarité avec l'embedding de la requête. Les paragraphes retournés sont ensuite concaténés pour former un contexte que le modèle Ollama utilisera pour générer une réponse.

In [ ]:
# Fonction pour extraire le contexte depuis Chroma
def extract_information_from_chroma(query, num_results=10):
    # Générer l'embedding pour la requête
    query_embedding = get_distilbert_embeddings(query)

    # Rechercher les documents les plus similaires dans Chroma
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=num_results
    )

    # Extraire les documents (sections)
    documents = results['documents']

    # Extraire le contexte
    context = ""
    for i in range(min(len(documents), num_results)):
        # Vérifier si le document est une liste de sections
        if isinstance(documents[i], list):
            # Concaténer toutes les sections du document
            context_text = " ".join(documents[i])
        else:
            context_text = documents[i]

        context += context_text + "\n\n"

    return context

###**8. Envoi de la requête à Ollama**

La requête est envoyée à Ollama via une API locale. En fonction du profil de l'utilisateur et du contexte extrait de Chroma, Ollama génère une réponse adaptée. Ce code assure également la gestion du flux de réponses, permettant une interaction en temps réel avec l'utilisateur.

In [ ]:
# Fonction pour interagir avec l'API d'Ollama via Docker
def query_ollama_with_context(context, query_text, profile):
    # Préparer le prompt avec le contexte
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context, query=query_text, profile=profile)

    # Envoyer la requête à l'API locale d'Ollama (qui doit tourner via Docker)
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "llama3.1", "prompt": prompt},
        stream=True  # Active le streaming
    )

    # Initialiser une variable pour stocker la réponse complète
    full_response = ""

    # Traiter chaque ligne de la réponse (streaming)
    for line in response.iter_lines():
        if line:
            # Décoder chaque ligne en JSON
            partial_response = line.decode('utf-8')
            try:
                # Convertir en JSON
                json_data = requests.models.complexjson.loads(partial_response)
                # Concaténer la partie 'response' du JSON à la réponse complète
                full_response += json_data.get('response', '')
            except ValueError as e:
                print(f"Erreur lors de la conversion JSON: {e}")
                print(f"Ligne brute : {partial_response}")

    # Afficher la réponse complète
    return full_response

In [ ]:
# Exemple d'utilisation
query = """
How can advanced techniques for optimizing systolic engines in FPGAs,
particularly through the use of DSP48E2 blocks in neural network architectures,
improve computational efficiency in embedded systems? Specifically,
how do techniques like weight prefetching in matrix systolic engines enhance
performance in FPGA accelerators such as Google's TPUv1 and Xilinx Vitis AI DPU,
which rely on specialized architectures to maximize parallelization and energy efficiency?
Additionally, what role do recent innovations in DSP48E2 multiplexing techniques,
ring accumulator design, and power consumption reduction play in providing state-of-the-art
solutions for complex neural systems? Lastly, how can these optimizations be applied
to integrate systolic engines in FPGA systems for neuromorphic applications,
including neural network-based accelerators and spiking neurons, with a focus on reducing
resource usage and improving overall computational performance?
"""

###**9. Mise en place de l’interface utilisateur avec Streamlit**

Cette partie met en place une interface utilisateur simple avec Streamlit :

- L’utilisateur peut choisir son profil (étudiant ou chercheur).
- Il peut poser une question en lien avec des articles scientifiques.
- Après soumission, le système extrait le contexte pertinent depuis Chroma et génère une réponse via Ollama, laquelle est affichée directement à l’écran.

In [ ]:
# Streamlit app
st.title("Scientific Query Assistant")

# Step 1: User profile selection
profile = st.selectbox("Select your profile:", ["student", "researcher"])

# Step 2: User enters a question
query = st.text_area("Enter your scientific question:")

# Step 3: Submit button
if st.button("Submit Query"):
    with st.spinner("Processing..."):
        context = extract_information_from_chroma(query)
        response_text = query_ollama_with_context(context, query, profile)
        st.write("### Response:")
        st.write(response_text)

##**Conclusion**

Ce notebook présente une approche complète pour créer un assistant intelligent capable de répondre à des questions scientifiques, en intégrant :

- Une base de données vectorielle (Chroma) pour stocker et interroger les textes.
- Un modèle de génération de réponses via Ollama, ajusté selon le profil de l'utilisateur.
- Une interface utilisateur simple et intuitive avec Streamlit.

L’architecture modulaire permet une extension facile et une adaptation pour d’autres domaines de recherche.